# Health Insurance Cross Sell Prediction Predictive Analysis

In [1]:
# Import packages
from data_loader import DataLoader
%matplotlib inline 

In [2]:
# Load data
train_url = "https://raw.githubusercontent.com/Mkho-cs/HICSP-Predictive-Analysis/main/dataset/train.csv"
test_url = "https://raw.githubusercontent.com/Mkho-cs/HICSP-Predictive-Analysis/main/dataset/test.csv"
train_data = DataLoader(train_url)
test_data = DataLoader(test_url)

## Data Transformation

In [3]:
# Data transformation starts here
"""
From the training data description, we can find features that are not important for training.
Hence, drop the following: id, Policy_Sales_Channel
"""
dropped = ['id', 'Policy_Sales_Channel']
train_data.drop_column(dropped)


In [4]:
train_data.display(10, True)


,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Vintage,Response
0,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,217,1
1,Male,76,1,3.0,0,1-2 Year,No,33536.0,183,0
2,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,27,1
3,Male,21,1,11.0,1,< 1 Year,No,28619.0,203,0
4,Female,29,1,41.0,1,< 1 Year,No,27496.0,39,0
5,Female,24,1,33.0,0,< 1 Year,Yes,2630.0,176,0
6,Male,23,1,11.0,0,< 1 Year,Yes,23367.0,249,0
7,Female,56,1,28.0,0,1-2 Year,Yes,32031.0,72,1
8,Female,24,1,3.0,1,< 1 Year,No,27619.0,28,0
9,Female,32,1,6.0,1,< 1 Year,No,28771.0,80,0


In [5]:
# Index categorical variables
"""
Categorical features have to be indexed to numerical by mapping them into dictionaries
Variables to map: Gender, Driving_License, Region_Code, Previously_Insured, Vehicle_Age, Vehicle_Damage, Response
"""

#First, check the types from default data loading
train_data.display_types()

Gender                 object
Age                     int64
Driving_License         int64
Region_Code           float64
Previously_Insured      int64
Vehicle_Age            object
Vehicle_Damage         object
Annual_Premium        float64
Vintage                 int64
Response                int64
dtype: object

In [8]:
# Change all the 'object' type columns to string
cols_to_string = ['Gender', 'Vehicle_Age', 'Vehicle_Damage']
train_data.column_to_string(cols_to_string)

# Change selected columns to categorical
cols_to_cat = ['Gender', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Response']
train_data.column_to_cat(cols_to_cat)
train_data.display_types()

Gender                category
Age                      int64
Driving_License       category
Region_Code           category
Previously_Insured    category
Vehicle_Age           category
Vehicle_Damage        category
Annual_Premium         float64
Vintage                  int64
Response              category
dtype: object

## Predictive Modelling

In [7]:
# Predictive modelling starts here